In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

import warnings
warnings.filterwarnings("ignore")

the dataset contains THREE YEARS of recorded incidents ("alarms") of broken fridges with some specific infos:
* When the fridge breaks down this is reported in the computer system and the CMS company sends an ingeneer to fix it and they it asks the Supermarket to pay for the service (maybe its a good business in Colombia ;)
* the original file has 150MB
* 600.000 records from 2012-2015

The questions are:
* If the company has details (data, logs etc) on all the "alarms" in the year 2012 - 2015, it wants to predicts how many alarms in each store in each month will happen in 2016?
* why?
* which shops has largest smallest number of alarms (maybe normalize per size of the shop)
* what data would you suggest the manager to collect to make the dataser more complete
* report and ask questions

Lets look at the data
* Work Order Number ----------------------------------------- 12939001 (like a factura, work done was stored as a number)
* Site Number -------------------------------------------------- A0052 (each store that company maintains has a number)
* Site Type ---------------------------------------------- SUPERMARKET (type of store: Supermarket(big)/Convenience(small))
* Date Created -------------------------------------- 02/02/2012 09:45 (date and time that the alarm occured)
* Date Completed  ----------------------------------- 02/02/2012 21:10 (date and time that the alarm was silenced)
* Provider Name  ------------------------------- CMS Solutions Limited (name of the maintenance company)
* WO Current Status ---------------------------------------------- EOL (I dont remember)
* Asset Category --------------------------------------- REFRIGERATION (type of asset, they also fix aircond, light etc)
* Problem Type Name ---------- Fixture Critical Over Temperature Alarm (type of alarm)
* Requested By ---------------------------- Urgent Refrigeration Alarm (priority of the alarm)
* WO Priority ------------------------------------------- Standard SLA (for accountancy of the company)
* Manufacturer ---------------------------------------------- LINDE AG (manufacturer of the asset that made the alarm)
* Asset In-Service Date ----------------------------- 23/05/2012 05:00 (when the machine was first connected)
* Level 1 Organization Name ------------------------------- Region S01 (the company divied the country into regions)
* Site Zipcode ----------------------------------------------- CR0 4XT (postal code of the store - helps to locate it)
* Sales Sq. Ft. ---------------------------------------------- 48409.0 (the size of the stor in square feet)
* Asset Barcode ------------------------------------------ SSL00319496 (number of the assed (fridge), a label)
* Client Product Name -------------------------------------------- NaN (dont remember)
* Asset Type ---------------------------------------- REMOTE MULTIDECK (fridge can be open, closed, cabinet etc)
* Asset Status ---------------------------------------------- Inactive (is the asset active today -time of generating dataset)

In [2]:
data = pd.read_csv("./data/datacl.csv")

In [7]:
data.head(3)

,cooling_range,preserves,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,...,requested_by,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,asset_type,asset_status
0,-18C - less,Frozen Food,12904684,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 10:00:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567391,HALF GLASS DOOR & WELL,In Service
1,-18C - less,Frozen Food,12904685,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 09:59:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567398,FULL GLASS DOOR,In Service
2,-18C - less,Frozen Food,12904853,A0892,SUPERMARKET,2012-01-02 05:10:00,2012-01-02 10:01:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567399,FULL GLASS DOOR,In Service


In [5]:
%%time
data["date_created"] = pd.DatetimeIndex(data.date_created)
data["date_completed"] = pd.DatetimeIndex(data.date_completed)
data["asset_in_service_date"] = pd.DatetimeIndex(data.asset_in_service_date)

CPU times: total: 266 ms
Wall time: 1.15 s


# ¿Cuantas alarmas ocurriran en el 2016 en cada tienda?
#### Se puede responder considerando cuando ocurre la alarma, es decir que la variable predictora (target) debe ser "date_created" y el resto de variables deben ser parametros caracteristicos (labels)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609989 entries, 0 to 609988
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   cooling_range              609989 non-null  object        
 1   preserves                  609989 non-null  object        
 2   work_order_number          609989 non-null  int64         
 3   site_number                609989 non-null  object        
 4   site_type                  609989 non-null  object        
 5   date_created               609989 non-null  datetime64[ns]
 6   date_completed             609989 non-null  datetime64[ns]
 7   provider_name              609989 non-null  object        
 8   wo_current_status          609989 non-null  object        
 9   asset_category             609989 non-null  object        
 10  problem_type_name          609989 non-null  object        
 11  requested_by               609989 non-null  object  

Seleccionamos las columnas que pueden servir para un primer modelo.
* predictora (target): date_created
* labels:
    * site_type (categorical)
    * date_completed (datetime)
    * provider_name (categorical)
    * asset_category (categorical)
    * problem_type_name (categorical)
    * requested_by (categorical)
    * wo_priority (categorical)
    * manufacturer (categorical) 
    * asset_in_service_date (datetime)
    * sales_sq_ft (categorical)
    * asset_type (float)
    * asset_status (categorical)
    * cooling_range (categorical)
    * preserves (categorical)

#### Convertir variables categoricas a numericas para implementarlas

In [15]:
target = data["date_created"]

In [9]:
data_filtered = data[["site_type","date_completed","provider_name","asset_category","problem_type_name",
               "requested_by","wo_priority","manufacturer","asset_in_service_date","sales_sq_ft",
               "asset_type","asset_status","cooling_range","preserves"]]
data_filtered.head()

,site_type,date_completed,provider_name,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,sales_sq_ft,asset_type,asset_status,cooling_range,preserves
0,SUPERMARKET,2012-01-02 10:00:00,Space Engineering services LTD,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,42639.0,HALF GLASS DOOR & WELL,In Service,-18C - less,Frozen Food
1,SUPERMARKET,2012-01-02 09:59:00,Space Engineering services LTD,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,42639.0,FULL GLASS DOOR,In Service,-18C - less,Frozen Food
2,SUPERMARKET,2012-01-02 10:01:00,Space Engineering services LTD,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,42639.0,FULL GLASS DOOR,In Service,-18C - less,Frozen Food
3,SUPERMARKET,2012-02-02 13:03:00,CMS Solutions Limited,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,LINDE,2002-01-08 05:00:00,13423.0,REMOTE MULTIDECK,In Service,3C - less,"Meat, Fish and Poultry"
4,SUPERMARKET,2012-08-02 19:39:00,CMS Solutions Limited,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,37870.0,FULL GLASS DOOR,Scrapped,No information,No information


## Usando factorize.
Si 3 categorias, una columna que lista las categorias en un rango

In [14]:
cat_columns = data_filtered.select_dtypes(['object']).columns
data_filtered[cat_columns] = data_filtered[cat_columns].apply(lambda x: pd.factorize(x)[0])
data_filtered.head()

,site_type,date_completed,provider_name,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,sales_sq_ft,asset_type,asset_status,cooling_range,preserves
0,0,2012-01-02 10:00:00,0,0,0,0,0,0,2011-01-06 15:19:00,42639.0,0,0,0,0
1,0,2012-01-02 09:59:00,0,0,0,0,0,0,2011-01-06 15:19:00,42639.0,1,0,0,0
2,0,2012-01-02 10:01:00,0,0,0,0,0,0,2011-01-06 15:19:00,42639.0,1,0,0,0
3,0,2012-02-02 13:03:00,1,0,0,0,0,1,2002-01-08 05:00:00,13423.0,2,0,1,1
4,0,2012-08-02 19:39:00,1,0,0,0,0,2,2009-03-15 04:00:00,37870.0,1,1,2,2


## Usando get_dummies.
Si 3 categorias, entonces [1,0,0],[0,1,0],[0,0,1] seria la transformacion (3 columnas para 3 categorias)

In [ ]:
column_encoded_site_type = pd.get_dummies(data["site_type"])
column_encoded_site_type.columns = ['{}_{}'.format("site_type", x) for x in column_encoded_site_type.columns]
column_encoded_site_type.head()

,site_type_CENTRAL LOCATION,site_type_CONVENIENCE STORE,site_type_DEPOT,site_type_PFS,site_type_SHARED SITE,site_type_STORE SUPPORT CENTRE,site_type_SUPERMARKET
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1


In [ ]:
column_encoded_provider_name = pd.get_dummies(data["provider_name"])
column_encoded_provider_name.columns = ['{}_{}'.format("provider_name", x) for x in column_encoded_provider_name.columns]
column_encoded_provider_name.head()

,provider_name_Airkool Contracts Ltd,provider_name_Allianz Engineering Inspection Services,provider_name_Bitzer UK,provider_name_CBES,provider_name_CMS Self Delivery,provider_name_CMS Solutions Limited,provider_name_Carillion (AMBS) Ltd,provider_name_Carter Refrigeration,provider_name_Closed Circuit Cooling (3CL),provider_name_Clover Technical Services Limited,...,provider_name_Parasense LTD,provider_name_Project 1st Engineering(Capital),provider_name_RCS ENERGY MANAGEMENT LTD,provider_name_RJ-AEL,provider_name_Ryan Jayberg Ltd,provider_name_Space Engineering services LTD,provider_name_TEST CONTRACTOR,provider_name_TRS Ltd.,provider_name_Verisae (RTMS System),provider_name_WR Refrigeration Ltd
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
column_encoded_asset_category = pd.get_dummies(data["asset_category"])
column_encoded_asset_category.columns = ['{}_{}'.format("asset_category", x) for x in column_encoded_asset_category.columns]
column_encoded_asset_category.head()

,asset_category_REFRIGERATION
0,1
1,1
2,1
3,1
4,1


In [53]:
column_encoded_problem_type_name = pd.get_dummies(data["problem_type_name"])
column_encoded_problem_type_name.columns = ['{}_{}'.format("problem_type_name", x) for x in column_encoded_problem_type_name.columns]
column_encoded_problem_type_name.head()

,problem_type_name_3CL Depot Monthly,problem_type_name_AEL Case Clean A4055 W/C 27/11/2012,problem_type_name_AEL Case clean A0171 A0038 A0119 W/C 16/04/2012,problem_type_name_AEL - Cabinet Maintenance and Clean 6 Monthly Retrospective A0875 A0661 A0679,problem_type_name_AEL - Cabinet Maintenance and Clean 6 Monthly W/C 05/03/2012,problem_type_name_AEL - Cabinet Maintenance and Clean 6 Monthly W/C 05/03/2012 visit 2,problem_type_name_AEL - Cabinet Maintenance and Clean 6 Monthly W/C 12/03/2012,problem_type_name_AEL - Cabinet Maintenance and Clean 6 Monthly W/C 12/03/2012 Visit 2,problem_type_name_AEL - Cabinet Maintenance and Clean 6 Monthly W/C 12/03/2012 Visit 3,problem_type_name_AEL - Cabinet Maintenance and Clean 6 Monthly W/C 13/02/2012,...,problem_type_name_WR - Refrigeration Controllers - Diligence - Nov 2012,problem_type_name_WR - Refrigeration Controllers - Diligence - Oct 2012,problem_type_name_WR - Refrigeration Gas Leak Checks - Legislative,problem_type_name_WR - Refrigeration Pressure Vessels -June 2012,problem_type_name_WR - Retarder Provers - Contractual Legislative,problem_type_name_WR - Retarder Provers - Legislative - Dec 2012,problem_type_name_WR - Retarder Provers - Legislative - Nov 2012,problem_type_name_WR - Retarder Provers - Legislative - Oct 2012,problem_type_name_WR - Retarder Provers - Legislative - Sep 2012,problem_type_name_WR - William Integral Cabinets - Nov 2012
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
column_encoded_requested_by = pd.get_dummies(data["requested_by"])
column_encoded_requested_by.columns = ['{}_{}'.format("requested_by", x) for x in column_encoded_requested_by.columns]
column_encoded_requested_by.head()

,requested_by_ AirOn/Off Temp Diff >8°C for 8hr within 24hr period(ProvCab) LIVE,requested_by_AEL Engineer,requested_by_ASL,requested_by_Air On/Off Temp Diff >10°C for 8hrs within 24hr period(Frozen Food Cabinets) LIVE,requested_by_Air On/Off Temp Diff >10°C for 12hrs within 24hr period(Frozen Food Cabinets) LIVE,requested_by_Air On/Off Temp Diff >10°C for 8hrs within 24hr period(Frozen Food Cabinets) LIVE,requested_by_Air On/Off Temp Diff >11°C for 12hrs within 24hr period(Meat and Poultry) LIVE,requested_by_Air On/Off Temp Diff >11°C for 8hrs within 24hr period(Fish Cabinets) LIVE,requested_by_Air On/Off Temp Diff >11°C for 8hrs within 24hr period(Meat and Poultry) LIVE,requested_by_Air On/Off Temp Diff >13°C for 12hrs within 24hr period(Meat and Poultry) LIVE,...,requested_by_Superheat >8°C for 2hrs sustained within 24hrs LIVE,requested_by_URGENT Refrigeration Fixture Communication Alarm,requested_by_URGENT Refrigeration Fixture Monitoring Alarm,requested_by_Urgent Refrigeration Alarm,requested_by_Urgent Refrigeration Alarm/K Desmond,requested_by_Valve Open >50% for 3hrs sustained for 3 times within 24hrs (All Cabinets and Coldrooms) LIVE,requested_by_Valve Open >50% for 3hrs sustained for 4 times within 24hrs (All Cabinets and Coldrooms) LIVE,requested_by_Valve Open >50% for 3hrs sustained for 4 times within 24hrs (All Cabinets and Coldrooms) LIVE,requested_by_georgia Boss Johnson,requested_by_store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [55]:
column_encoded_wo_priority = pd.get_dummies(data["wo_priority"])
column_encoded_wo_priority.columns = ['{}_{}'.format("wo_priority", x) for x in column_encoded_wo_priority.columns]
column_encoded_wo_priority.head()

,wo_priority_PPM,wo_priority_PPM Remedial,wo_priority_Predictive,wo_priority_Standard SLA
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


In [56]:
column_encoded_manufacturer = pd.get_dummies(data["manufacturer"])
column_encoded_manufacturer.columns = ['{}_{}'.format("manufacturer", x) for x in column_encoded_manufacturer.columns]
column_encoded_manufacturer.head()

,manufacturer_ACES,manufacturer_ADANDE,manufacturer_ADVANSOR,manufacturer_ADVANTECH,manufacturer_AHT,manufacturer_AIRKOOL,manufacturer_ALAN NUTHALL LTD,manufacturer_ALAN NUTTALL,manufacturer_ALAN NUTTALL HINCKLEY,manufacturer_ALAN NUTTALL LTD,...,manufacturer_TYLER,manufacturer_UNKNOWN,manufacturer_VALERA,manufacturer_VERCO,manufacturer_VERCO LTD,manufacturer_VISCOUNT CATERING LTD,manufacturer_WILLIAMS,manufacturer_WILLIAMS REFRIGERATION,manufacturer_WR REFRIGERATION,manufacturer_ZANOTTI
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
column_encoded_sales_sq_ft = pd.get_dummies(data["sales_sq_ft"])
column_encoded_sales_sq_ft.columns = ['{}_{}'.format("sales_sq_ft", x) for x in column_encoded_sales_sq_ft.columns]
column_encoded_sales_sq_ft.head()

,sales_sq_ft_868.0,sales_sq_ft_916.0,sales_sq_ft_1073.0,sales_sq_ft_1085.0,sales_sq_ft_1090.0,sales_sq_ft_1164.0,sales_sq_ft_1167.0,sales_sq_ft_1171.0,sales_sq_ft_1179.0,sales_sq_ft_1181.0,...,sales_sq_ft_67049.0,sales_sq_ft_67252.0,sales_sq_ft_67336.0,sales_sq_ft_70227.0,sales_sq_ft_71267.0,sales_sq_ft_71496.0,sales_sq_ft_72605.0,sales_sq_ft_77436.0,sales_sq_ft_82853.0,sales_sq_ft_87176.0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
column_encoded_asset_status = pd.get_dummies(data["asset_status"])
column_encoded_asset_status.columns = ['{}_{}'.format("asset_status", x) for x in column_encoded_asset_status.columns]
column_encoded_asset_status.head()

,asset_status_In Service,asset_status_Inactive,asset_status_No information,asset_status_Not Applicable,asset_status_Not Found,asset_status_Not Verified,asset_status_Planned Refit Removal,asset_status_Scrapped,asset_status_Sold,asset_status_Storage - Hold,asset_status_Storage - Needs Review,asset_status_Storage - Operable
0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0


In [59]:
column_encoded_cooling_range = pd.get_dummies(data["cooling_range"])
column_encoded_cooling_range.columns = ['{}_{}'.format("cooling_range", x) for x in column_encoded_cooling_range.columns]
column_encoded_cooling_range.head()

,cooling_range_(2-8)C - less,cooling_range_(5-12)C - less,cooling_range_-18C - less,cooling_range_3C - less,cooling_range_5C - less,cooling_range_7C - less,cooling_range_No information
0,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1


In [60]:
column_encoded_preserves = pd.get_dummies(data["preserves"])
column_encoded_preserves.columns = ['{}_{}'.format("preserves", x) for x in column_encoded_preserves.columns]
column_encoded_preserves.head()

,preserves_Frozen Food,"preserves_Meat, Fish and Poultry",preserves_No information,preserves_Pharmacy,preserves_Produce,"preserves_Provisions, Dairy and Prepared Produce",preserves_Soft Drinks/BWS
0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0


In [65]:
list_encoded = [column_encoded_site_type,
column_encoded_provider_name,
column_encoded_asset_category,
column_encoded_problem_type_name,
column_encoded_requested_by,
column_encoded_wo_priority,
column_encoded_manufacturer,
column_encoded_sales_sq_ft,
column_encoded_asset_status,
column_encoded_cooling_range,
column_encoded_preserves]

In [93]:
data = data.join(list_encoded[0])
data = data.join(list_encoded[1])
data = data.join(list_encoded[2])
data = data.join(list_encoded[3])
data = data.join(list_encoded[4])
data = data.join(list_encoded[5])
data = data.join(list_encoded[6])
data = data.join(list_encoded[7])
data = data.join(list_encoded[8])
data = data.join(list_encoded[9])
data = data.join(list_encoded[10])

In [100]:
data.head()

,cooling_range,preserves,work_order_number,site_number,site_type,date_created,date_completed,provider_name,wo_current_status,asset_category,...,cooling_range_5C - less,cooling_range_7C - less,cooling_range_No information,preserves_Frozen Food,"preserves_Meat, Fish and Poultry",preserves_No information,preserves_Pharmacy,preserves_Produce,"preserves_Provisions, Dairy and Prepared Produce",preserves_Soft Drinks/BWS
0,-18C - less,Frozen Food,12904684,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 10:00:00,Space Engineering services LTD,EOL,REFRIGERATION,...,0,0,0,1,0,0,0,0,0,0
1,-18C - less,Frozen Food,12904685,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 09:59:00,Space Engineering services LTD,EOL,REFRIGERATION,...,0,0,0,1,0,0,0,0,0,0
2,-18C - less,Frozen Food,12904853,A0892,SUPERMARKET,2012-01-02 05:10:00,2012-01-02 10:01:00,Space Engineering services LTD,EOL,REFRIGERATION,...,0,0,0,1,0,0,0,0,0,0
3,3C - less,"Meat, Fish and Poultry",12927899,A0416,SUPERMARKET,2012-01-02 08:40:00,2012-02-02 13:03:00,CMS Solutions Limited,EOL,REFRIGERATION,...,0,0,0,0,1,0,0,0,0,0
4,No information,No information,12932231,A0003,SUPERMARKET,2012-01-02 13:33:00,2012-08-02 19:39:00,CMS Solutions Limited,EOL,REFRIGERATION,...,0,0,1,0,0,1,0,0,0,0


In [105]:
to_erase = ['cooling_range', 'preserves','work_order_number', 'site_number','work_order_number', 'site_number','provider_name',
       'wo_current_status', 'asset_category', 'problem_type_name','requested_by', 'wo_priority', 'manufacturer',
       'level_1_organization_name', 'site_zipcode', 'sales_sq_ft','asset_barcode','asset_status']

In [110]:
data.to_csv("./data/data_dummies.csv",index=None)

In [ ]:
data.drop(to_erase[0],axis=1,inplace=True)
data.drop(to_erase[1],axis=1,inplace=True)
data.drop(to_erase[2],axis=1,inplace=True)
data.drop(to_erase[3],axis=1,inplace=True)
data.drop(to_erase[4],axis=1,inplace=True)
data.drop(to_erase[5],axis=1,inplace=True)
data.drop(to_erase[6],axis=1,inplace=True)
data.drop(to_erase[7],axis=1,inplace=True)
data.drop(to_erase[8],axis=1,inplace=True)
data.drop(to_erase[9],axis=1,inplace=True)
data.drop(to_erase[10],axis=1,inplace=True)
data.drop(to_erase[11],axis=1,inplace=True)
data.drop(to_erase[12],axis=1,inplace=True)
data.drop(to_erase[13],axis=1,inplace=True)
data.drop(to_erase[14],axis=1,inplace=True)
data.drop(to_erase[15],axis=1,inplace=True)
data.drop(to_erase[16],axis=1,inplace=True)
data.drop(to_erase[17],axis=1,inplace=True)

Problema de este es que aunque seria mas eficiente para el algoritmo de machine learning, resulta en una tabla de peso 6 GB, lo que genera problemas para hacer otras conversiones. 